# Order and Delivery to Google Earth Engine

In this notebook we're going to cover how to order data using the [Orders API](https://developers.planet.com/apis/orders/) and deliver it to [Google Earth Engine (GEE)](https://earthengine.google.com/) using [Planet’s GEE Delivery Integration](https://developers.planet.com/docs/integrations/gee/).

This example demonstrates how to:
1. Define the data to be ordered
2. Build a cloud delivery configuration object (`delivery_config`), which tells the Orders API where to deliver the data
3. Build an order request to be sent off to the Orders API, `iowa_order`
4. Create the order and have it deliver to your GEE project

**Prerequisites:** 
- [Planet's Python SDK 2.0](https://github.com/planetlabs/planet-client-python) installed and initialized in your environment. Please follow instructions in [our docs](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/get-started/quick-start-guide/) to authenticate your account with Planet servers. (**This notebook is not compatible with earlier versions of Planet's Python SDK**)
- An AOI - `iowa_aoi`
- Item ID(s) - `iowa_images`
- A GEE project with EE API enabled - `planet-devrel-dev`
- A pre-existing GEE [ImageCollection](https://developers.google.com/earth-engine/guides/ic_creating) - `gee-integration-testing`
- An account with a download quota. Not sure if you have download quota? Please [get in touch](https://www.planet.com/contact-sales/).

In [1]:
import planet

## Define data to be ordered

In [2]:
# The area of interest (AOI) defined as a polygon
iowa_aoi = {
    "type":
    "Polygon",
    "coordinates": [[[-91.198465, 42.893071], [-91.121931, 42.893071],
                     [-91.121931, 42.946205], [-91.198465, 42.946205],
                     [-91.198465, 42.893071]]]
}

# The item IDs we wish to order
iowa_images = ['20200925_161029_69_2223', '20200925_161027_48_2223']

## Define cloud delivery location
This is the *key concept* of this notebook. Here, we are defining a cloud delivery configuration object, `cloud_config`, where we are defining the destination to be Google Earth Engine, in the project named `planet-devrel-dev`, in the ImageCollection named `gee-integration-testing`.

In [3]:
# Google Earth Engine configuration
cloud_config = planet.order_request.google_earth_engine(
    project='planet-devrel-dev', collection='gee-integration-testing')
# Order delivery configuration
delivery_config = planet.order_request.delivery(cloud_config=cloud_config)

## Build a simple order request
Here we are building an order request, where we specify the products we wish to order. Spesifically, here we are requesting the IDs defined in `iowa_images`, as `analytic_udm2` assets, from the Planet Scope Scene product, `PSScene`.

In [4]:
# Product description for the order request
data_products = [
    planet.order_request.product(item_ids=iowa_images,
                                 product_bundle='analytic_sr_udm2',
                                 item_type='PSScene')
]

# Build the order request
iowa_order = planet.order_request.build_request(name='iowa_order',
                                                products=data_products,
                                                delivery=delivery_config)

print(iowa_order)

{'name': 'iowa_order', 'products': [{'item_ids': ['20200925_161029_69_2223', '20200925_161027_48_2223'], 'item_type': 'PSScene', 'product_bundle': 'analytic_sr_udm2'}], 'delivery': {'google_earth_engine': {'project': 'planet-devrel-dev', 'collection': 'gee-integration-testing'}}}


## Create and deliver the order
Define a function that will create an order and update you with its progress. In this case, since we specified the delivery destination to be Google Earth Engine with `delivery_config`, it will also wait for it to be delivered to your project.

In [5]:
async def create_and_deliver_order(order_request, client):
    '''Create and deliver an order.

    Parameters:
        order_request: An order request
        client: An Order client object
    '''
    with planet.reporting.StateBar(state='creating') as reporter:
        # Place an order to the Orders API
        order = await client.create_order(order_request)
        reporter.update(state='created', order_id=order['id'])
        # Wait while the order is being completed
        await client.wait(order['id'],
                          callback=reporter.update_state,
                          max_attempts=0)

    # Grab the details of the orders
    order_details = await client.get_order(order_id=order['id'])

    return order_details

Create and deliver the order, `iowa_order`, to the GEE project, `planet-devrel-dev`, in the ImageCollection, `gee-integration-testing`.

In [6]:
async with planet.Session() as ps:
    # The Orders API client
    client = ps.client('orders')
    # Create the order and deliver it to GEE
    order_details = await create_and_deliver_order(iowa_order, client)

54:18 - order befd89fa-d314-4194-9001-1ac656fc8d27 - state: success


Display the results for the first item data in the order.

In [7]:
print(order_details['_links']['results'][0])

{'delivery': 'success', 'expires_at': '2022-12-01T00:37:11.420Z', 'location': 'https://api.planet.com/compute/ops/download/?token=eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE2Njk4NTUwMzEsInN1YiI6IllzV0ZqTDE4c04zeXYyZE1HQmNjeEVjeDFCQ0ZHeVgrUmlZWG9UYkU1Q3lFL2lMblE2VHhVMzBMc3JuczR6aEI5TUZvTmVTc0pHUFEzQ3lkc1hRMDVRPT0iLCJ0b2tlbl90eXBlIjoiZG93bmxvYWQtYXNzZXQtc3RhY2siLCJhb2kiOiIiLCJhc3NldHMiOlt7Iml0ZW1fdHlwZSI6IiIsImFzc2V0X3R5cGUiOiIiLCJpdGVtX2lkIjoiIn1dLCJ1cmwiOiJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY29tcHV0ZS1vcmRlcnMtbGl2ZS8yMDIwMDkyNV8xNjEwMjlfNjlfMjIyM18zQl9BbmFseXRpY01TX1NSLnRpZj9YLUdvb2ctQWxnb3JpdGhtPUdPT0c0LVJTQS1TSEEyNTZcdTAwMjZYLUdvb2ctQ3JlZGVudGlhbD1jb21wdXRlLWdjcy1zdmNhY2MlNDBwbGFuZXQtY29tcHV0ZS1wcm9kLmlhbS5nc2VydmljZWFjY291bnQuY29tJTJGMjAyMjExMzAlMkZhdXRvJTJGc3RvcmFnZSUyRmdvb2c0X3JlcXVlc3RcdTAwMjZYLUdvb2ctRGF0ZT0yMDIyMTEzMFQwMDM3MTFaXHUwMDI2WC1Hb29nLUV4cGlyZXM9ODYzOTlcdTAwMjZYLUdvb2ctU2lnbmF0dXJlPWE2OWQxNjE4ZGMwZDk0ODY3Y2I2N2EzY2U3MGUyNTU0M2VhODMyMmU2ZTFmZjFjMjE1NWVlZGU4O

## Build an order request with clipping and harmonization applied
Here we are building an order request similar to the one above, but with [clipping](https://developers.planet.com/apis/orders/tools/#clip) and [harmonization](https://developers.planet.com/apis/orders/tools/#harmonization) applied. Namely, we are asking the Orders API to clip our images to the AOI, as defined by `iowa_aoi`, and radiometrically harmonizing the imagery with Sentinel-2's sensor. It's important to note that although the Orders API supports more tools, clipping and harmonization are the only two that are currently supported with GEE integration.

In [8]:
# Clip images to the AOI's perimeter and harmonize the data with Dove Classic
tools = [
    planet.order_request.clip_tool(iowa_aoi),
    planet.order_request.harmonize_tool('Sentinel-2')
]

# Build the order request
iowa_order = planet.order_request.build_request(name='iowa_order',
                                                products=data_products,
                                                delivery=delivery_config,
                                                tools=tools)

print(iowa_order)

{'name': 'iowa_order', 'products': [{'item_ids': ['20200925_161029_69_2223', '20200925_161027_48_2223'], 'item_type': 'PSScene', 'product_bundle': 'analytic_sr_udm2'}], 'delivery': {'google_earth_engine': {'project': 'planet-devrel-dev', 'collection': 'gee-integration-testing'}}, 'tools': [{'clip': {'aoi': {'type': 'Polygon', 'coordinates': [[[-91.198465, 42.893071], [-91.121931, 42.893071], [-91.121931, 42.946205], [-91.198465, 42.946205], [-91.198465, 42.893071]]]}}}, {'harmonize': {'target_sensor': 'Sentinel-2'}}]}


Create and deliver the clipped and harmonized data to `planet-devrel-dev` in `gee-integration-testing`.

In [9]:
async with planet.Session() as ps:
    # The Orders API client
    client = ps.client('orders')
    # Create the order and deliver it to GEE
    order_details = await create_and_deliver_order(iowa_order, client)

08:16 - order 37d82131-6a36-4b5c-a651-df30ca8ef945 - state: success


Display the results for the first item data in the order.

In [10]:
print(order_details['_links']['results'][0])

{'delivery': 'success', 'expires_at': '2022-12-01T01:05:06.668Z', 'location': 'https://api.planet.com/compute/ops/download/?token=eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE2Njk4NTY3MDYsInN1YiI6IjdyZW9xVXRQaUQ5WVhaZkpwRU1wUkZQZVVBeFVzcDU2c3NaSnBTN0dMV3RZcWtZSFZJYWhwZXRISER6MHgxZTBTSWVYQk9CWjYrUXRDeDZFYnREVFlRPT0iLCJ0b2tlbl90eXBlIjoiZG93bmxvYWQtYXNzZXQtc3RhY2siLCJhb2kiOiIiLCJhc3NldHMiOlt7Iml0ZW1fdHlwZSI6IiIsImFzc2V0X3R5cGUiOiIiLCJpdGVtX2lkIjoiIn1dLCJ1cmwiOiJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY29tcHV0ZS1vcmRlcnMtbGl2ZS8yMDIwMDkyNV8xNjEwMjlfNjlfMjIyM18zQl91ZG0yX2NsaXAudGlmP1gtR29vZy1BbGdvcml0aG09R09PRzQtUlNBLVNIQTI1Nlx1MDAyNlgtR29vZy1DcmVkZW50aWFsPWNvbXB1dGUtZ2NzLXN2Y2FjYyU0MHBsYW5ldC1jb21wdXRlLXByb2QuaWFtLmdzZXJ2aWNlYWNjb3VudC5jb20lMkYyMDIyMTEzMCUyRmF1dG8lMkZzdG9yYWdlJTJGZ29vZzRfcmVxdWVzdFx1MDAyNlgtR29vZy1EYXRlPTIwMjIxMTMwVDAxMDUwNlpcdTAwMjZYLUdvb2ctRXhwaXJlcz04NjM5OVx1MDAyNlgtR29vZy1TaWduYXR1cmU9YTExNGE3Mjk3ZTNhZmEwYzRmNmU2ZDQxYjRkNzgxMWQ1YTkwNTcxNjI0MmMxMDUzZWI2OGYxZjE3NjhiY